<a href="https://colab.research.google.com/github/darkfapper/Pandoras_Box/blob/main/Bayesian_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import pandas as pd
import matplotlib.pyplot as plt
tfd = tfp.distributions
tfb = tfp.bijectors
tfn = tfp.experimental.nn

NUM_CLASSES = 10
NUM_TRAIN_EXAMPLES = 100



# Load in the data
cifar10 = tf.keras.datasets.mnist

# Distribute it to train and test set
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# x_train = x_train.astype("float32") /255.
# x_test = x_test.astype("float32") / 255.

x_train =np.expand_dims(x_train, -1).astype("float32") / 255.
x_test =np.expand_dims(x_test, -1).astype("float32") / 255.


kl_divergence_function = (lambda q, p, _: tfd.kl_divergence(q, p) /  tf.cast(x_train.shape[0], dtype=tf.float32))

In [ ]:
x_train.shape

In [ ]:

## Fonte: https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/bayesian_neural_network.py
model = tf.keras.models.Sequential([
      tf.keras.layers.Input((28,28,1)),
      tfp.layers.Convolution2DFlipout(
          6, kernel_size=5, padding='SAME',
          kernel_divergence_fn=kl_divergence_function,
          activation=tf.nn.selu),
      tf.keras.layers.MaxPooling2D(
          pool_size=[2, 2], strides=[2, 2],
          padding='SAME'),
      tfp.layers.Convolution2DFlipout(
          32, kernel_size=5, padding='SAME',
          kernel_divergence_fn=kl_divergence_function,
          activation=tf.nn.selu),
      tf.keras.layers.MaxPooling2D(
          pool_size=[2, 2], strides=[2, 2],
          padding='SAME'),
      tfp.layers.Convolution2DFlipout(
          120, kernel_size=5, padding='SAME',
          kernel_divergence_fn=kl_divergence_function,
          activation=tf.nn.selu),
      tf.keras.layers.Flatten(),
      tfp.layers.DenseFlipout(
          84, kernel_divergence_fn=kl_divergence_function,
          activation=tf.nn.selu),
      tfp.layers.DenseFlipout(
          NUM_CLASSES, kernel_divergence_fn=kl_divergence_function,
          activation=tf.nn.softmax)
  ])


adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = adam, loss = "sparse_categorical_crossentropy",metrics = ["accuracy"])
model.summary()

In [ ]:

# ## Fonte: https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/bayesian_neural_network.py
# model = tf.keras.models.Sequential([
#       tf.keras.layers.Input((32,32,3)),
#       tfp.layers.Convolution2DFlipout(
#           32, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.selu),
#       tf.keras.layers.MaxPooling2D(
#           pool_size=[2, 2], strides=[2, 2],
#           padding='SAME'),
#       tfp.layers.Convolution2DFlipout(
#           32, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.selu),
#       tf.keras.layers.MaxPooling2D(
#           pool_size=[2, 2], strides=[2, 2],
#           padding='SAME'),
#       tfp.layers.Convolution2DFlipout(
#           64, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.selu),
#       tfp.layers.Convolution2DFlipout(
#           64, kernel_size=5, padding='SAME',
#           kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.selu),
#       tf.keras.layers.MaxPooling2D(
#           pool_size=[2, 2], strides=[2, 2],
#           padding='SAME'),
#       tf.keras.layers.Flatten(),
#       tfp.layers.DenseFlipout(
#           2048, kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.selu),
#       tfp.layers.DenseFlipout(
#           1024, kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.selu),
#       tfp.layers.DenseFlipout(
#           NUM_CLASSES, kernel_divergence_fn=kl_divergence_function,
#           activation=tf.nn.softmax)
#   ])


# adam = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer = adam, loss = "sparse_categorical_crossentropy",metrics = ["accuracy"])
# model.summary()

In [ ]:
history = model.fit(x_train, y_train, epochs = 10,validation_data = (x_test, y_test))

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
lista = []
NUM_MC_SAMPLES = 300
for i in range(NUM_MC_SAMPLES):
  lista.append(model(x_test))

In [ ]:
imarray = np.random.rand(28,28,1) * 255

In [ ]:
import matplotlib.pyplot as plt
arr = np.array(lista)

x = np.random.randint(0, x_test.shape[0])
val = arr[:,x,:]
img = x_test[x,:]
img =imarray
plt.imshow(img, cmap=plt.get_cmap('gray'))
plt.show()
val = pd.DataFrame(val)
val.hist( bins=25,
grid=False, figsize=(15,12), color='#86bf91',rwidth=0.9)
plt.show()

In [ ]:
from PIL import Image
img = Image.fromarray(x_train[999,:],"RGB")
img.save("img.png")
img.show()